## Extract Data from Pardot and Save it to a CSV for Further Processing
- This is a context in which data is pulled from the Salesforce API and saved as a CSV.
- There are a number of packages required.

In [1]:
#!pip3 install simple_salesforce plotly pymongo urllib3 oauth2client httplib2 datetime

In [2]:
#This imports the configuration and sets the files. 
import configparser
cfg = configparser.ConfigParser()
#cfg.read('../../config_dev.ini')
cfg.read('../../config_prd.ini')
file_marker='_all'



In [3]:
from datetime import datetime
from pytz import timezone
tz = timezone('America/Chicago')
now = datetime.now(tz) 
dateAPI=now.strftime('%Y-%m-%d %H:%M')
dateAPI

'2017-02-13 23:09'

In [4]:
import pickle
import pandas as pd
pd.set_option("display.max_rows",999)
pd.set_option("display.max_columns",999)
import datetime
import logging
#This connects to the mongoDB
import pymongo
import cgi
import requests
import json
from sqlalchemy import create_engine


In [4]:

mongouri=cfg.get('Mongo', 'mongouri')
mongodb=cfg.get('Mongo', 'mongodb')
home= cfg.get('Default', 'home')
#postgres_uri= cfg.get('Postgres', 'postgres_uri')
on_docker= cfg.get('Default', 'on_docker')
salesforce_email= cfg.get('Salesforce', 'salesforce_email')
pardot_email= cfg.get('Salesforce', 'pardot_email')
pardot_password= cfg.get('Salesforce', 'pardot_password')
pardot_user_key= cfg.get('Salesforce', 'pardot_user_key')
pardot_api_base= cfg.get('Salesforce', 'pardot_api_base')


In [5]:
#engine = create_engine(postgress_uri)
today = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
today = '_'+today
logger = logging.getLogger('Salesforce')
hdlr = logging.FileHandler('../../logs/salesforce.log')
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr) 
logger.setLevel(logging.INFO)



In [6]:
def logIt(message):
    logger.info(message)
    print(message)

In [7]:

# Connection to Mongo DB
try:
    conn=pymongo.MongoClient(mongouri)
    db = conn[mongodb]
    logIt("Connection to MongoDB was successful.")
except:
    logIt("Connection to MongoDB failed.", db)
    #This retreives the User Tokens.


Connection to MongoDB was successful.


In [8]:
#This retreives the User Tokens.
try:
    user = db.users.find_one({'email':salesforce_email})
    access_token=user['tokens'][0]['accessToken']['params']['access_token']
    refresh_token=user['tokens'][0]['refreshToken']
    instance_url=user['tokens'][0]['accessToken']['params']['instance_url']
    logIt("Retrieved User Token." )
except: 
    logIt("Token retreival not successful.")

Retrieved User Token.


In [9]:
#This gets a api_key.
data = {
	   'email': pardot_email,
	   'password': pardot_password,
	   'user_key': pardot_user_key,
       'format': 'json'
}

try:
    response = requests.post(pardot_api_base,data=data).json()
    api_key=response['api_key']
    logIt('Retrieved updated api_key.')
except:
    logIt('Error retrieving updated api_key.', response)


Retrieved updated api_key.


In [ ]:
/api/email/version/3/do/stats/id/<email id>

In [20]:
def pardotObject(objectName,offset,updated_after=None,emailId=None):
    if emailId!=None:
        extension='/query'
    else:
        extension='/query'
    
    
    api_url='https://pi.pardot.com/api/'+objectName+'/version/3/do'+extension
    data = {
	   'api_key': api_key,
	   'user_key': pardot_user_key,
       'format': 'json',
       'output': 'bulk',
       'offset': offset
    }
    if updated_after!=None:
        data['updated_after']=updated_after
    if email!=None:
        data['email']=email


    try:
        logIt('Trying URL: '+api_url)
        
        return requests.post(api_url,data=data).json()
        
    except:
        logIt('Error retrieving data.', response)
        return "error"



In [21]:
def pardotQuery(objectName,offset=0, updated_after=None, maxApiCalls=None,emailId=None):
    file=home+'data/csv/production/'+pardot_email+'-'+'pardot'+'-'+objectName+today+".csv"
    out = open ( file , 'w')
    response=pardotObject(objectName, offset,updated_after,email)
    if response['result']!=None:
        df=pd.DataFrame(response['result'][objectName])
        df.to_csv(out, header=True)
        offset+=len(df.index)
        apiCalls=1
        logIt('Records Retreived: '+str(offset)+' in '+str(apiCalls)+' api calls.\n')
    while response['result']!=None and apiCalls<maxApiCalls:
        response=pardotObject(objectName, offset,updated_after,email)
        if response['result']!=None:
            df=pd.DataFrame(response['result'][objectName])
            df.to_csv(out, header=False)
            offset+=len(df.index)
            apiCalls+=1
            logIt('Records Retreived: '+str(offset)+' in '+str(apiCalls)+' api calls.\n')
        else:
            logIt('Finished')
    out.close()


In [22]:
pardotQuery(objectName='prospect', maxApiCalls=3)

Trying URL: https://pi.pardot.com/api/prospect/version/3/do/query
Records Retreived: 200 in 1 api calls.

Trying URL: https://pi.pardot.com/api/prospect/version/3/do/query
Records Retreived: 400 in 2 api calls.

Trying URL: https://pi.pardot.com/api/prospect/version/3/do/query
Records Retreived: 600 in 3 api calls.



In [23]:
pardotQuery(objectName='user', maxApiCalls=3)

Trying URL: https://pi.pardot.com/api/user/version/3/do/query
Records Retreived: 36 in 1 api calls.

Trying URL: https://pi.pardot.com/api/user/version/3/do/query
Finished


In [24]:
pardotQuery(objectName='campaign', maxApiCalls=3)

Trying URL: https://pi.pardot.com/api/campaign/version/3/do/query
Records Retreived: 158 in 1 api calls.

Trying URL: https://pi.pardot.com/api/campaign/version/3/do/query
Finished


In [25]:
#Dynamic Content isn't being used.
pardotQuery(objectName='dynamicContent', maxApiCalls=3)

Trying URL: https://pi.pardot.com/api/dynamicContent/version/3/do/query
Records Retreived: 2 in 1 api calls.

Trying URL: https://pi.pardot.com/api/dynamicContent/version/3/do/query
Finished


In [26]:
pardotQuery(objectName='email', maxApiCalls=3,email='denise_stones@whhs.com')

Trying URL: https://pi.pardot.com/api/email/version/3/do/query


KeyError: 'result'

In [27]:
response

{'@attributes': {'stat': 'ok', 'version': 1},
 'api_key': 'a95f9c4a0ce5502c9f4edb0939f7f4a7'}